In [203]:
import sys
!{sys.executable} -m pip install urllib

  ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


In [204]:
import requests
import lxml.html as lh
import pandas as pd

In [205]:
#Let's see what the output for one month looks like:
year = '2020'
month = '01' #adapt for for loop
location = 'great-hill-ma'

#This code (next 3 lines) is adapted from https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
url='https://www.usharbors.com/harbor/massachusetts/'+location+'/tides/?tide='+year+'-'+month+'#monthly-tide-chart'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#save the raw html rows as "table"
table = tr_elements

In [206]:
#print the lengths of the first few rows to see where the actual tide chart starts
print("\nThe number of columns in the first twelve rows are:")
print([len(T) for T in table[:12]])

#check the output of one row
print("\nSample row output:")
row = 10
for col in range(len(table[row])):
    print(table[row][col].text_content())
print("Doesn't look like we need the last 3 columns. So we'll just grab the first 12.")
number_of_columns = 12
    
#Check how many rows we have
print("\nThe number of rows is: " + str(len(table)))


The number of columns in the first twelve rows are:
[1, 4, 6, 10, 15, 15, 15, 15, 15, 15, 15, 15]

Sample row output:
7
Tue
5:08
4.2
5:34
3.4
11:00
0.6
10:38
0.4
7:08
4:29

7
Tue
Doesn't look like we need the last 3 columns. So we'll just grab the first 12.

The number of rows is: 37


In [207]:
#Function which scrapes the URL for the current month
def get_month_table(month):
    if month > 9: #month as a string to use for the URL def
        month_str = str(month)
    else: 
        month_str = '0'+str(month)
    #This code (next 3 lines) is adapted from https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
    url='https://www.usharbors.com/harbor/massachusetts/great-hill-ma/tides/?tide='+year_str+'-'+month_str+'#monthly-tide-chart'
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')
    #save the raw html rows as "table"
    table = tr_elements
    return(table)

In [208]:
def find_first_row(table):
    for row in range(len(table)):
        if table[row][0].text_content() == '1':
            return(row)

In [209]:
def isInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [210]:
months = ['January','February','March','April','May','June','July','August','September','October','November','December']

In [217]:
year_str = '2020'

#Empty df to store the data:
df = []

for month in range(1,13):
    month_str_display = months[month-1]
    table = get_month_table(month) #get the data for this month

    #find the first row
    row = find_first_row(table)

    #while the first entry of the row is less than the number of days in the month
    while isInt(table[row][0].text_content()):
        row_data = ['']*number_of_columns
        for col in range(number_of_columns):
            row_data[col] = table[row][col].text_content()
        row_data.insert(0,month_str_display)
        df.append(row_data)
        row+=1

In [218]:
print("The number of rows is "+str(len(df))+", this number should match the number of days in the given year, so 365 or 366")


The number of rows is 366, this number should match the number of days in the given year, so 365 or 366


In [219]:
df

[['January',
  '1',
  'Wed',
  '12:00',
  '3.6',
  '12:20',
  '3.6',
  '4:57',
  '0.7',
  '5:21',
  '0.6',
  '7:08',
  '4:24'],
 ['January',
  '2',
  'Thu',
  '12:53',
  '3.6',
  '1:14',
  '3.4',
  '5:45',
  '0.8',
  '6:05',
  '0.7',
  '7:08',
  '4:25'],
 ['January',
  '3',
  'Fri',
  '1:44',
  '3.7',
  '2:04',
  '3.3',
  '6:43',
  '1.0',
  '6:57',
  '0.7',
  '7:08',
  '4:25'],
 ['January',
  '4',
  'Sat',
  '2:32',
  '3.8',
  '2:55',
  '3.2',
  '7:51',
  '1.0',
  '7:53',
  '0.8',
  '7:08',
  '4:26'],
 ['January',
  '5',
  'Sun',
  '3:23',
  '3.9',
  '3:48',
  '3.2',
  '9:04',
  '0.9',
  '8:50',
  '0.7',
  '7:08',
  '4:27'],
 ['January',
  '6',
  'Mon',
  '4:16',
  '4.0',
  '4:43',
  '3.2',
  '10:12',
  '0.8',
  '9:47',
  '0.6',
  '7:08',
  '4:28'],
 ['January',
  '7',
  'Tue',
  '5:08',
  '4.2',
  '5:34',
  '3.4',
  '11:00',
  '0.6',
  '10:38',
  '0.4',
  '7:08',
  '4:29'],
 ['January',
  '8',
  'Wed',
  '5:55',
  '4.4',
  '6:21',
  '3.5',
  '11:38',
  '0.3',
  '11:24',
  '0.2',
  '7:

In [220]:
column_names = ['Month','Date','Day','high_1','height_hi_1','high_2','height_hi_2','low_1','height_low_1','low_2','height_low_2','sunrise','sunset']
data = pd.DataFrame(df,columns=column_names)

In [221]:
data

,Month,Date,Day,high_1,height_hi_1,high_2,height_hi_2,low_1,height_low_1,low_2,height_low_2,sunrise,sunset
0,January,1,Wed,12:00,3.6,12:20,3.6,4:57,0.7,5:21,0.6,7:08,4:24
1,January,2,Thu,12:53,3.6,1:14,3.4,5:45,0.8,6:05,0.7,7:08,4:25
2,January,3,Fri,1:44,3.7,2:04,3.3,6:43,1.0,6:57,0.7,7:08,4:25
3,January,4,Sat,2:32,3.8,2:55,3.2,7:51,1.0,7:53,0.8,7:08,4:26
4,January,5,Sun,3:23,3.9,3:48,3.2,9:04,0.9,8:50,0.7,7:08,4:27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,December,27,Sun,5:56,4.2,6:20,3.4,12:18 PM,0.6,11:26,0.6,7:07,4:20
362,December,28,Mon,6:37,4.3,7:01,3.5,12:23 PM,0.5,11:59,0.4,7:07,4:21
363,December,29,Tue,7:17,4.4,7:40,3.6,,,12:50,0.4,7:07,4:22
364,December,30,Wed,7:56,4.5,8:20,3.6,12:35,0.3,1:26,0.3,7:08,4:23


In [222]:
data.to_csv('tide-chart-'+location+'-'+year_str+'.csv',index=False)